# Computational Problem Set 1
Jeanne Sorin

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
import time
from scipy import sparse

In [ ]:
r = 0.05
δ = 1/(1+r)
Stot = 1000

## 1. Solving the DDP by value function iteration ; discrete state space

In [ ]:
N = 501
Na = 501
S = np.linspace(0,1000,N)
A = np.linspace(0,1000,Na)

def u_f1(S,y):
    U = 2*y**0.5
    if y > S:
        U = -100000000
    return U

def u_f2(S,y):
    U = 5*y - 0.05*y**2
    if y > S:
        U = -100000000
    return U

U = np.ones((N, Na))

In [ ]:
for i in range(0,N):
    for j in range(0,Na):
        U[i,j] = u_f1(S[i], A[j])        

In [ ]:
Index = np.ones((N, Na))
for i in range(0,N):
    for j in range(0,Na):
        Index[i,j] = max(i-j, 0.0)

In [ ]:
N = 1000
Na = 1000
T = np.zeros((N, N*Na))
for i in range(0, N):
    for k in range(0, N):
        for j in range(0, Na):
            T[i,k*Na+j] = (i - j == k)

In [ ]:
T_sparse = np.copy(T)
T_sparse = csr_matrix(T_sparse)

In [ ]:
T_sparse

In [ ]:
# V(S) = U(S, a) + δV(S-a)
#(T_sparse==1).nonzero()
#[i.nonzero() for i in (T_sparse==1)]
list_T = [i.nonzero()[1].tolist() for i in (T_sparse==1)]

In [ ]:
list_T[1][:]

In [ ]:
N = 100
Na = 100
Vorig = np.zeros((N, Na))
Vnext = np.copy(Vorig)
Pol = np.copy(A)

In [ ]:
for i in range(0,N):
    print(i)
    for j in range(0,Na):
        index = max(i-j,0)
        Vnext[i,j] = u_f1(S[i], Pol[i]) + δ*Vorig[index, int(Pol[i])]
    Pol[i] = np.argmax(Vnext[i,:])

In [ ]:
Pol

In [ ]:
Vnext[3,]

In [ ]:
# Initialize the value and control functions, and solve the model using value function iteration
S_tot = 1000
V_init = 0
A_init = 0



In [ ]:
I = np.array([0,3,1,0])
J = np.array([0,3,1,2])
V = np.array([4,5,7,9])
A = sparse.coo_matrix((V,(I,J)),shape=(4,4))
print(A)

#### b) Set up the action space 

In [ ]:

def Clumsy_VFI(Kgrid, V0, maxiter, tol, 
               δ=(1/1.05)., 
              printerror="False", 
              graph="True",
              smartk="False",
              smartstoring="False",
              howard="False"):
    '''
    Skeleton function to compute the Value Function and the Policy Function for the simple extraction model.
    Arg
    - ygrid: the grid of potential values of y (array)
    - V0: the initial guess for the value function (array)
    - maxiter: the maximum number of iterations for the VFI algorithm (scalar)
    - tol: the error tolerance (scalar)
    - δ = 1/1.05 default
    
    Example:
    Clumsy_VFI(ygrid, V0, 1000, 1e-5, δ = 0.96)
    '''
    
    ts = time.time()
    
    ### Initiate vectors & numbers
    iter = 0
    epsi = 1
    Kp_new = np.zeros(len(Kgrid))
    Vp_new = np.zeros(len(Kgrid))
    gk = np.zeros(len(Kgrid))
    Vp = V0
    
    
    ### SMARTSTORING: compute the utility function matrix beforehand
    utilitymatrix = np.zeros((len(Kgrid), len(Kgrid)))
    if smartstoring == "True":
        for i in range(len(Kgrid)):
            for j in range(len(Kgrid)):
                utilitymatrix[i,j] = utility(z*Kgrid[i]**(α) + (1-δ)*Kgrid[i] - Kgrid[j]) 

                
    ### BEGINNING OF THE LOOP FOR THE VFI           
    while (epsi > tol) & (iter < maxiter):
        for ik in range(n):
            
            ### SMART K GRID: specify the minimum k for the policy function
            if smartk == "True":
                if ik==0:
                    d = 0
                else:
                    d = int(gk[ik-1])
            else:
                d = 0
                
            
            ### Compute the Value function Tj for each choice of g(k)
            Tj = np.zeros(len(Kgrid)-d) # Create vector & feed it
            for jk in range(d, len(Kgrid)):
                    Tj[jk-d] = Bellman(Kgrid, Vp, ik, jk, utilitymatrix, smartstoring)
            
            ### Choose the maximum
            Vp_new[ik] = np.max(Tj)
            Kp_new[ik] = d + int(np.where(Tj==max(Tj))[0])
         
        ### HOWARD
        if howard=="True":
            for c in range(100):
                for ik in range(len(Kgrid)):
                    jk = int(Kp_new[ik])
                    Vp_new[ik] = Bellman(Kgrid, Vp_new, ik, jk, utilitymatrix, smartstoring)

        ### Check the error
        epsi = np.abs(Vp_new - Vp).max()
        
        ### Keep track of what is going on
        if printerror == "True":
            print("Error term: ", epsi) 
        if epsi < tol:
            break
        
        Kp = Kp_new*1. #Update
        Vp = Vp_new*1. #Update
        iter=iter+1
        
    tf = time.time()
    time_final = tf - ts
            
    #Calculate Values for g(k)
    for jk in range(len(Kgrid)):
        gk[jk] = g_hat(Kgrid, int(Kp[jk]))
        
        
    if graph == "True":
        plt.plot(Kgrid, gk)
        plt.axhline(y=kss, color='r', linestyle='-')
        plt.xlabel("k")
        plt.ylabel("g(k)")
        plt.title("Policy Function")
        plt.show()
        plt.plot(Kgrid, Vp)
        plt.xlabel("k")
        plt.ylabel("V(k)")
        plt.title("Value Function")
        plt.show()
        print("Time is ", time_final)
        print("Number of iterations is ", iter)


    return[Vp, gk, iter, time_final]
        

In [ ]:
ygrid = np.copy(A)
V0 = np.zeros(len(ygrid))
iter = 0
epsi = 1
yp_new = np.zeros(len(ygrid))
Vp_new = np.zeros(len(ygrid))
gk = np.zeros(len(ygrid))
Vp = V0
    
utilitymatrix = U   

In [ ]:
# ik = index of the stock today
ik = 0
# d = max extraction < current stock
#d = int(gk[max(ik-1), 0])
d = 0

#Value function Tj for each choice g(k)
Tj = np.zeros(len(ygrid)-d)

# Compute the value function Tj for each choice of g(k)
for jk in range(d, len(ygrid)):
    Tj[jk - d] = Bellman(ygrid, Vp, ik, jk, utilitymatrix)
    
# Choose the maximum
Vp_new[ik] = np.max(Tj)
Kp_new[ik] = d + int(np.where(Tj==max(Tj))[0])

### BEGINNING OF THE LOOP FOR THE VFI           
#while (epsi > tol) & (iter < maxiter):
    #for ik in range(n):
            


### Check the error
#epsi = np.abs(Vp_new - Vp).max()
'''
#print("Error term: ", epsi) 
if epsi < tol:
    break
Kp = Kp_new*1. #Update
Vp = Vp_new*1. #Update
iter=iter+1
    

#Calculate Values for g(k)
for jk in range(len(Kgrid)):
    gk[jk] = g_hat(Kgrid, int(Kp[jk]))
'''



# B. Code

After preparing the environment, I define a list of functions that will be called inside the main function, Clumsy_VFI in order to solve a simple neoclassical growth model.
Functions are described below.

Because loops and matrices are fairly interchangeable in Python, I did not write this code with the same concerns for loops one may have when coding in Matlab.


Nb: Clumsy_VFI received this name because I made sure not to use the efficient scipy.optimize optimizers, interpolant or smart k grids.

In [ ]:
'''
Define Functions
'''
# The Utility Function
def utility(c):
    '''
    Function that computes the utility for a given level of consumption according to the log utility functional form.
    Penalty on negative consumption.
    Arg: 
    - c : consumption
    '''
    if c > 0:
        U = np.log(c)
    if c <= 0:
        U = -9999999999
    return U

# The steady state
def find_K_SS(α, β, δ, z):
    '''
    Function computing the k steady state for parameters α, β, δ and z (model specific)
    Arg
    - α : power of k in the production function f(k) = z k^α
    - β : discount factor
    - δ : depreciation rate
    - z : from the production function f(k) = z k^α
    '''
    kss = (δ/(α*β*z))**(1/(α-1))
    return(kss)

# The Bellman Equation
def Bellman(Kgrid, Vp, ik, jk, 
            utilitymatrix, 
            smartstoring="False"):
    '''
    Function computing the value function by adding current period's utility of consumption, and next period's discounted value function
    Account for options utilitymatrix and smartstoring.
    Arg:
    - Kgrid : the grid of k values (array)
    - Vp : potential value functions (array)
    - ik : index of today's k in Kgrid (int)
    - jk : index of tomorrow's k (g(k)) in Kgrid (int)
    - smartstoring = "False" : whether or not using utility computed beforehand
    - utilitymatrix : Corresponding utility matrix 
    '''
    if smartstoring=="False":
        Value = utility(z*Kgrid[ik]**(α) + (1-δ)*Kgrid[ik] - Kgrid[jk])  + β * Vp[jk]   
    else:
        Value = utilitymatrix[ik, jk]  + β * Vp[jk]   
    return Value

def g_hat(Kgrid, jk):
    '''
    Function computing the resulting policy function g_hat.
    Arg
    - Kgrid: the grid of potential ks.
    - jk: the index corresponding to the policy function.
    '''
    policy = α*β*Kgrid[jk]**α
    return(policy)
β = 0.98
α = 1/3
δ = 1.
z = 1.

def Clumsy_VFI(kss, Kgrid, V0, maxiter, tol, 
               α=1/3, β=0.98, δ=1., z=1.,
              printerror="False", 
              graph="True",
              smartk="False",
              smartstoring="False",
              howard="False"):
    '''
    Skeleton function to compute the Value Function and the Policy Function for the simple Neoclassical Growth Model.
    Arg
    - kss: the value for the k at the steady state (scalar)
    - Kgrid: the grid of potential values of k (array)
    - V0: the initial guess for the value function (array)
    - maxiter: the maximum number of iterations for the VFI algorithm (scalar)
    - tol: the error tolerance (scalar)
    - α = 1/3 default
    - β = 0.98 default
    - δ = 1. default
    - z = 1. default
    - printerror="False" whether or not printing the error at each iteration
    - graph="True" whether or not output graphs of the value and policy functions
    - smartk="False" whether or not choosing K in a smart way using the increasing property of the policy function
    - smartstoring="False" whether or not compute a matrix of utility beforehand
    - howard="False" whether or not run the howard algorith
    
    Example:
    Clumsy_VFI(kss, Kgrid, V0, 1000, 1e-5, α=)
    '''
    
    ts = time.time()
    
    ### Initiate vectors & numbers
    iter = 0
    epsi = 1
    Kp_new = np.zeros(len(Kgrid))
    Vp_new = np.zeros(len(Kgrid))
    gk = np.zeros(len(Kgrid))
    Vp = V0
    
    
    ### SMARTSTORING: compute the utility function matrix beforehand
    utilitymatrix = np.zeros((len(Kgrid), len(Kgrid)))
    if smartstoring == "True":
        for i in range(len(Kgrid)):
            for j in range(len(Kgrid)):
                utilitymatrix[i,j] = utility(z*Kgrid[i]**(α) + (1-δ)*Kgrid[i] - Kgrid[j]) 

                
    ### BEGINNING OF THE LOOP FOR THE VFI           
    while (epsi > tol) & (iter < maxiter):
        for ik in range(n):
            
            ### SMART K GRID: specify the minimum k for the policy function
            if smartk == "True":
                if ik==0:
                    d = 0
                else:
                    d = int(gk[ik-1])
            else:
                d = 0
                
            
            ### Compute the Value function Tj for each choice of g(k)
            Tj = np.zeros(len(Kgrid)-d) # Create vector & feed it
            for jk in range(d, len(Kgrid)):
                    Tj[jk-d] = Bellman(Kgrid, Vp, ik, jk, utilitymatrix, smartstoring)
            
            ### Choose the maximum
            Vp_new[ik] = np.max(Tj)
            Kp_new[ik] = d + int(np.where(Tj==max(Tj))[0])
         
        ### HOWARD
        if howard=="True":
            for c in range(100):
                for ik in range(len(Kgrid)):
                    jk = int(Kp_new[ik])
                    Vp_new[ik] = Bellman(Kgrid, Vp_new, ik, jk, utilitymatrix, smartstoring)

        ### Check the error
        epsi = np.abs(Vp_new - Vp).max()
        
        ### Keep track of what is going on
        if printerror == "True":
            print("Error term: ", epsi) 
        if epsi < tol:
            break
        
        Kp = Kp_new*1. #Update
        Vp = Vp_new*1. #Update
        iter=iter+1
        
    tf = time.time()
    time_final = tf - ts
            
    #Calculate Values for g(k)
    for jk in range(len(Kgrid)):
        gk[jk] = g_hat(Kgrid, int(Kp[jk]))
        
        
    if graph == "True":
        plt.plot(Kgrid, gk)
        plt.axhline(y=kss, color='r', linestyle='-')
        plt.xlabel("k")
        plt.ylabel("g(k)")
        plt.title("Policy Function")
        plt.show()
        plt.plot(Kgrid, Vp)
        plt.xlabel("k")
        plt.ylabel("V(k)")
        plt.title("Value Function")
        plt.show()
        print("Time is ", time_final)
        print("Number of iterations is ", iter)


    return[Vp, gk, iter, time_final]
        

In [ ]:
'''
Preliminary
'''
### Parameters
β = 0.98
α = 1/3
δ = 1.
z = 1.

### kss
kss = find_K_SS(α, β, δ, z)
k0 = 0.05*kss

### Make Grid
k1 = 0.05*kss
kn = 1.2*kss 
n = 200

Kgrid = np.linspace(k1, kn, n)
V0 = np.zeros(n)
tol=1e-5

In [ ]:
print("kss is ", kss)
print("Kgrid min is ", Kgrid[0])
print("Kgrid max is ", Kgrid[n-1])

# B. Brute Force Value Function Iteration

In [ ]:
[Vp_B, Kp_B, iter_B, time_final_B] = Clumsy_VFI(kss, Kgrid, V0, 
                                                5000, tol, graph = "True")

# C. An Improved in Guess $V_0$

In [ ]:
V0_new = np.zeros(n)
for ik in range(n):
    V0_new[ik] = utility(z*Kgrid[ik]**α - δ*Kgrid[ik])/(1-β)

[Vp_C, Kp_C, iter_C, time_final_C] = Clumsy_VFI(kss, Kgrid, V0_new, 50, tol,
                                               graph="True")



# D. An Improved Decision Process

In [ ]:
[Vp_D, Kp_D, iter_D, time_final_D] = Clumsy_VFI(kss, Kgrid, 
                                                V0, 
                                                5000, 
                                                tol,
                                                smartk="True")

[Vp_Dp, Kp_Dp, iter_Dp, time_final_Dp] = Clumsy_VFI(kss, Kgrid, 
                                                V0_new, 
                                                100, 
                                                tol,
                                                graph="False",
                                                smartk="True")

# E. Storing the Return Function

In [ ]:
[Vp_E, Kp_E, iter_E, time_final_E] = Clumsy_VFI(kss, Kgrid, 
                                                V0, 
                                                5000, 
                                                tol,
                                                smartstoring="True")

[Vp_Ep, Kp_Ep, iter_Ep, time_final_Ep] = Clumsy_VFI(kss, Kgrid, 
                                                V0_new, 
                                                5000, 
                                                tol,
                                                smartk="True",
                                                smartstoring="True")

# F. Howard’s Improvement algorithm: Fewer maximizations

In [ ]:
[Vp_F, Kp_F, iter_F, time_final_F] = Clumsy_VFI(kss, Kgrid,
                                                V0, 
                                                5000, 
                                                tol,
                                                howard="True",
                                                graph="True")

# G. Combining Methods

In [ ]:
[Vp_G, Kp_G, iter_G, time_final_G] = Clumsy_VFI(kss, Kgrid,
                                                V0_new, 
                                                100, 
                                                tol,
                                                smartk="True",
                                                smartstoring="True",
                                                howard="True",
                                                graph="True")

In [ ]:
print("Time in seconds:")
print("Time Brute Force: ", round(time_final_B, 3))
print("Time Improved Guess: ", round(time_final_C, 3))
print("Time Improved Decision Process: ", round(time_final_D, 3))
print("Time Improved Decision Process + Improved Guess: ", round(time_final_Dp, 3)) 
print("Time Storing the utility function: ", round(time_final_E, 3))
print("Time Storing the utility function + Improved Guess + Improved Decision: ", round(time_final_Ep, 3))
print("Time Howard: ", round(time_final_F, 3))
print("Time Howard + Improved Guess + Improved Decision + Storage: ", round(time_final_G, 3))

Running the different algorithms I show that the brute force algorithm is more than 2 orders of magnitude slower than any of the optimized algorithms using the improved guess V0, which seems to be the main source for improving the algorithm.

The Howard algorithm by itself does better than the improved decision process or storing the utility function alone, but not as good as the improved guess alone.

Moreover, comparing "storing the utility function + improved guess + improved decision" versus "storing the utility function + improved guess + improved decision + Howard" (0.426 vs 0.457) suggests that if the algorithm is improved beforehand, Howard does not improve the fit. Besides, by imposing 100 loops for Howard, we add running time, which seems to be unnecessary in this case.

In conclusion, Howard or an improved initial guess appear to be the "cheapest" optimizations to improve your code with. This conclusion holds for Python, but may be different for a language like Matlab, where matrices play a role, or like Julia.